In [1]:
import gradio as gr
import cv2
import numpy as np

# Function to convert 2x3 affine matrix to 3x3 for matrix multiplication
def to_3x3(affine_matrix):
    return np.vstack([affine_matrix, [0, 0, 1]])

# Function to apply transformations based on user inputs
def apply_transform(image, scale, rotation, translation_x, translation_y, flip_horizontal):

    # 将 PIL 格式的图像转换为 NumPy 数组
    image = np.array(image)
    
    # 为了避免边界问题，对图像进行填充
    pad_size = min(image.shape[0], image.shape[1]) // 2
    image_new = np.zeros((pad_size*2+image.shape[0], pad_size*2+image.shape[1], 3), dtype=np.uint8) + np.array((255,255,255), dtype=np.uint8).reshape(1,1,3)
    image_new[pad_size:pad_size+image.shape[0], pad_size:pad_size+image.shape[1]] = image
    image = np.array(image_new)
    
    # 初始化变换后的图像
    transformed_image = np.array(image)
    
    ### 实现复合变换
    # 注意：对于缩放和旋转，需要围绕图像中心进行

    # 获取图像中心坐标
    center = (image.shape[1] / 2, image.shape[0] / 2)
    
    # 生成旋转和缩放矩阵（2x3），围绕图像中心
    rot_mat = cv2.getRotationMatrix2D(center, rotation, scale)
    # 将 2x3 的矩阵转换为 3x3，用于矩阵乘法
    rot_mat_3x3 = to_3x3(rot_mat)
    
    # 生成平移矩阵（3x3）
    trans_mat = np.array([
        [1, 0, translation_x],
        [0, 1, translation_y],
        [0, 0, 1]
    ], dtype=np.float32)
    
    # 生成水平翻转矩阵（3x3）
    if flip_horizontal:
        flip_mat = np.array([
            [-1, 0, image.shape[1]],  # 围绕垂直轴进行翻转
            [0, 1, 0],
            [0, 0, 1]
        ], dtype=np.float32)
    else:
        flip_mat = np.eye(3, dtype=np.float32)  # 单位矩阵（不进行翻转）
    
    # 组合所有的变换矩阵
    # 注意矩阵乘法的顺序，右乘表示先进行的变换
    # 变换顺序：先翻转 -> 缩放和旋转 -> 平移
    total_mat = trans_mat @ rot_mat_3x3 @ flip_mat  # 矩阵乘法
    
    # 从 3x3 矩阵中取出前两行，得到 2x3 的仿射变换矩阵
    final_mat = total_mat[:2, :]
    
    # 使用 cv2.warpAffine 应用变换
    transformed_image = cv2.warpAffine(image, final_mat, (image.shape[1], image.shape[0]), borderValue=(255,255,255))
    
    return transformed_image


# Gradio Interface
def interactive_transform():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Transformation Playground")
        
        # Define the layout
        with gr.Row():
            # Left: Image input and sliders
            with gr.Column():
                image_input = gr.Image(type="pil", label="Upload Image")

                scale = gr.Slider(minimum=0.1, maximum=2.0, step=0.1, value=1.0, label="Scale")
                rotation = gr.Slider(minimum=-180, maximum=180, step=1, value=0, label="Rotation (degrees)")
                translation_x = gr.Slider(minimum=-300, maximum=300, step=10, value=0, label="Translation X")
                translation_y = gr.Slider(minimum=-300, maximum=300, step=10, value=0, label="Translation Y")
                flip_horizontal = gr.Checkbox(label="Flip Horizontal")
            
            # Right: Output image
            image_output = gr.Image(label="Transformed Image")
        
        # Automatically update the output when any slider or checkbox is changed
        inputs = [
            image_input, scale, rotation, 
            translation_x, translation_y, 
            flip_horizontal
        ]

        # Link inputs to the transformation function
        image_input.change(apply_transform, inputs, image_output)
        scale.change(apply_transform, inputs, image_output)
        rotation.change(apply_transform, inputs, image_output)
        translation_x.change(apply_transform, inputs, image_output)
        translation_y.change(apply_transform, inputs, image_output)
        flip_horizontal.change(apply_transform, inputs, image_output)

    return demo

# Launch the Gradio interface
interactive_transform().launch()

/mnt/store/zys/anaconda3/lib/python3.11/site-packages/gradio_client/documentation.py:105: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/mnt/store/zys/anaconda3/lib/python3.11/site-packages/gradio_client/documentation.py:105: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
